# Day 8: Haunted Wasteland

You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about ghosts a few minutes ago.

One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of network of labeled nodes.

It seems like you're meant to use the left/right instructions to navigate the network. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: AAA and ZZZ. You feel like AAA is where you are now, and you have to follow the left/right instructions until you reach ZZZ.

This format defines each node of the network individually. For example:

```
RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
```

Starting with AAA, you need to look up the next element based on the next left/right instruction in your input. In this example, start with AAA and go right (R) by choosing the right element of AAA, CCC. Then, L means to choose the left element of CCC, ZZZ. By following the left/right instructions, you reach ZZZ in 2 steps.

Of course, you might not find ZZZ right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: RL really means RLRLRLRLRLRLRLRL... and so on. For example, here is a situation that takes 6 steps to reach ZZZ:

```
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
```

Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?



In [33]:
import regex as re
from itertools import cycle


p_test = re.compile(r"(?P<from>[A-Z]{3}) = \((?P<left>[A-Z]{3}), (?P<right>[A-Z]{3})\)")
path = ""
mapping = {}
with open("2023_day_8_input.txt") as f:
    for i, line in enumerate(f):
        if i == 0:
            path = line[:-1]
            continue
        if line == "\n":
            continue
        match = p_test.match(line)
        if match is not None:
            mapping[match.group("from")] = {
                "L": match.group("left"),
                "R": match.group("right"),
            }

pos = "AAA"
step = 0
for dir in cycle(path):
    pos = mapping[pos][dir]
    step += 1
    if pos == "ZZZ":
        break
step

16697

# Part Two

The sandstorm is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take significantly more steps to escape!

What if the map isn't for people - what if the map is for ghosts? Are ghosts even bound by the laws of spacetime? Only one way to find out.

After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in A is equal to the number ending in Z! If you were a ghost, you'd probably just start at every node that ends with A and follow all of the paths at the same time until they all simultaneously end up at nodes that end with Z.

For example:

```
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
```

Here, there are two starting nodes, 11A and 22A (because they both end with A). As you follow each left/right instruction, use that instruction to simultaneously navigate away from both nodes you're currently on. Repeat this process until all of the nodes you're currently on end with Z. (If only some of the nodes you're on end with Z, they act like any other node and you continue as normal.) In this example, you would proceed as follows:

- Step 0: You are at 11A and 22A.
- Step 1: You choose all of the left paths, leading you to 11B and 22B.
- Step 2: You choose all of the right paths, leading you to 11Z and 22C.
- Step 3: You choose all of the left paths, leading you to 11B and 22Z.
- Step 4: You choose all of the right paths, leading you to 11Z and 22B.
- Step 5: You choose all of the left paths, leading you to 11B and 22C.
- Step 6: You choose all of the right paths, leading you to 11Z and 22Z.
So, in this example, you end up entirely on nodes that end in Z after 6 steps.

Simultaneously start on every node that ends with A. How many steps does it take before you're only on nodes that end with Z?

In [84]:
import numpy as np

pos = [p for p in mapping.keys() if p.endswith("A")]
step = 0
for dir in cycle(path):
    pos = [mapping[p][dir] for p in pos]
    step += 1
    if np.all([p.endswith("Z") for p in pos]):
        break
    if step == 100000:
        print("Solution not found")
        break

Solution not found


This is apparently not the right way to find the answer. Let's check for the cycling behavior. Starting from a point we must necessarily reach a point where we will just loop. The lower bound for this loop length being the length of the cycling set of instructions (if its minimal, let's assume that)

In [76]:
cycle_mapping = {}

for start in mapping.keys():
    pos = start
    step = 0
    end_steps = []
    for dir in path:
        pos = mapping[pos][dir]
        step += 1
        if pos.endswith("Z"):
            end_steps.append(step)
    cycle_mapping[start] = {
        "end_steps": end_steps,
        "next": pos,
    }

In [79]:
{p: cycle_mapping[p] for p in cycle_mapping.keys() if len(cycle_mapping[p]["end_steps"]) > 0}

{'XDB': {'end_steps': [283], 'next': 'KKZ'},
 'MXJ': {'end_steps': [283], 'next': 'XSZ'},
 'MFR': {'end_steps': [283], 'next': 'BLZ'},
 'RVX': {'end_steps': [283], 'next': 'RGZ'},
 'BVX': {'end_steps': [283], 'next': 'KKZ'},
 'RFP': {'end_steps': [283], 'next': 'RGZ'},
 'LSS': {'end_steps': [283], 'next': 'VQZ'},
 'MQR': {'end_steps': [283], 'next': 'BLZ'},
 'BGT': {'end_steps': [283], 'next': 'ZZZ'},
 'HFQ': {'end_steps': [283], 'next': 'ZZZ'},
 'LPS': {'end_steps': [283], 'next': 'VQZ'},
 'MMR': {'end_steps': [283], 'next': 'XSZ'}}

After mapping each node to the corresonding node a the end of the instruction cycl we can also observe that end nodes can only be reached at the end of cycle of instruction. Which is useful for us

In [108]:
from itertools import count

pos = [p for p in mapping.keys() if p.endswith("A")]
for c in count():
    pos = [cycle_mapping[p]["next"] for p in pos]
    if np.all([p.endswith("Z") for p in pos]):
        break
    if c == 100000:
        print("Solution not found")
        break
c

Solution not found


100000

Still unsuccessful. Let's check for the cycles of each start points

In [111]:
starts = [p for p in mapping.keys() if p.endswith("A")]
cycle_lens = []
for start in starts:
    pos_list = [start]
    end_cycle = []
    for c in count(1):
        new_pos = cycle_mapping[pos_list[-1]]["next"]
        if new_pos.endswith("Z"):
            end_cycle.append(c)
        if new_pos in pos_list:
            print(pos_list.index(new_pos))
            print(len(pos_list))
            print(end_cycle)
            cycle_lens.append(len(pos_list) - pos_list.index(new_pos))
            break
        pos_list.append(new_pos)

1
72
[71]
1
44
[43]
1
48
[47]
1
74
[73]
1
60
[59]
1
62
[61]


Another simplification comes from the fact that all start points reach a cycle after the first sequence of instructions. Also these cycles only contains a node ending with "Z" just before the end of the cycle. This means that the cycles will reach an end node at the same time when the cycles will end together. The answeris thus as follow..

In [114]:
from math import lcm

lcm(*cycle_lens) * len(path)

10668805667831